# Finding and Ordering Imagery for the Largest Active Fire on Earth

In this notebook, we'll combine NASA's FIRMS (Fire Information for Resource Management System) active fire data with Google Earth Engine and Planet imagery to:

1. **Load today's active fires** from the MODIS or VIIRS instruments via FIRMS
2. **Identify the largest fire** by grouping nearby fire pixels into connected clusters
3. **Polygonize the fire boundary** to create an area of interest (AOI)
4. **Search Planet's catalog** for recent PlanetScope imagery over that AOI
5. **Order and visualize** the imagery directly in this notebook using geemap

This workflow demonstrates real-world remote sensing analysis: detecting an event from coarse hotspot data, defining a precise AOI, and acquiring high-resolution commercial imagery for detailed assessment.

## APIs and Tools

- **[Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install)** – for geospatial analysis and data processing
- **[geemap](https://geemap.org/)** – interactive mapping library for Earth Engine in Python
- **[Planet SDK for Python](https://planet-sdk-for-python-v2.readthedocs.io/)** – for programmatic access to Planet's data catalog
- **[Planet Orders API](https://developers.planet.com/docs/apis/orders/)** – for ordering and delivering Planet imagery

## Datasets

- **[FIRMS Active Fire Data](https://firms.modaps.eosdis.nasa.gov/)** – near real-time fire detections from MODIS and VIIRS instruments
- **[PlanetScope Imagery](https://developers.planet.com/docs/data/planetscope/)** – high-resolution (3–5 m) daily imagery from Planet's satellite constellation

## Prerequisites

- Authenticated access to Google Earth Engine (`ee.Authenticate()` and `ee.Initialize()`)
- A Planet API key (sign up at [planet.com](https://www.planet.com/) for a free trial or educational account)
- Python packages: `earthengine-api`, `geemap`, `planet`, `requests`, `geopandas`, `shapely`

Let's get started!

## Installing Required Libraries

Before we begin, we need to install the necessary Python packages for this workflow:

- **earthengine-api** – Google Earth Engine Python API for geospatial analysis
- **geemap** – interactive mapping library that simplifies Earth Engine visualization
- **planet** – Planet SDK for searching and ordering commercial satellite imagery
- **requests** – HTTP library for API calls (e.g., downloading FIRMS data)
- **geopandas** – geospatial extension of pandas for working with vector data
- **shapely** – geometric operations library for creating and manipulating spatial objects

Run the cell below to install or upgrade these packages. The `-q` flag suppresses verbose output, and `--upgrade` ensures you have the latest versions.

**Note:** Some operations in this workflow (particularly Planet API interactions) may benefit from asynchronous execution to handle multiple requests efficiently.

In [1]:
!pip install -q --upgrade earthengine-api leafmap geemap planet requests geopandas shapely async 

## 1. Authentication with Google Earth Engine

Before you can use GEE, you need to authenticate your account. There are several ways to do this:

*   **Browser Authentication (Recommended):** This is the easiest method.  GEE will open a browser window to allow you to log in with your Google account.
*   **Service Account:**  More suitable for automated scripts and server-side applications. Requires creating a service account in the Google Cloud Console and downloading credentials.

### 1.1 Browser Authentication (using `ee.Initialize()`)

The following code snippet initializes the Earth Engine API and authenticates your account using browser authentication.  Run this cell *first* to authenticate.

In [2]:
import ee
import geemap as geemap

# Initialize Earth Engine (authenticate if needed)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()
# Print a confirmation message to let us know the initialization was successful
print('Earth Engine initialized.')

Earth Engine initialized.


In [7]:

# Load the FIRMS image collection.
dataset = ee.ImageCollection("FIRMS")

geometry = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Get the most recent image (for visualization)
lastimg = dataset.sort('system:time_start', False).first()

# Select the T21 band from images in the last 7 days
fires = lastimg.select('T21').gt(100)

# Compute the number of connected pixels (including itself) in each cluster.
connectedCount = fires.connectedPixelCount(1000, True)

# Visualization styles
firesVis = {
    "min": 100.0,
    "max": 500.0,
    "palette": ["yellow", "orange", "red"]
}

countVis = {
    "min": 0.0,
    "max": 100.0,
    "palette": ["yellow", "orange", "red"]
}

# Calculate the min and max values for each band in the region.
minMax = connectedCount.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=geometry,
    scale=1000,  # Set an appropriate scale for your analysis
    maxPixels=1e9  # Increase maxPixels if needed for large regions
)

# Print the result to the console.
print("Min and Max values:", minMax.getInfo())



Min and Max values: {'T21_max': 254, 'T21_min': 2}


In [8]:

# Visualization styles
firesVis = {
    "min": 100.0,
    "max": 500.0,
    "palette": ["yellow", "orange", "red"]
}

countVis = {
    "min": 0.0,
    "max": 100.0,
    "palette": ["yellow", "orange", "red"]
}

# Visualization: purple -> yellow ramp for connected pixel counts
conn_vis = {
    'min': 0,
    'max': 100,
    'palette': ['#4B0082', '#6A5ACD', '#8A2BE2', '#DA70D6', '#FFD700']
}



m = geemap.Map(center=[0, 0], zoom=2)
m.addLayer(lastimg.select('T21'), firesVis, 'FIRMS T21')

# Add the connectedCount layer to the existing geemap Map
m.addLayer(connectedCount, conn_vis, 'Connected pixels (count) — purple→yellow')

# Display the map (ensure this is the last line in the cell)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [11]:
T21_max = ee.Number(minMax.get('T21_max'))
largest_cluster = connectedCount.eq(T21_max)
largest_mask = largest_cluster.updateMask(largest_cluster)
m.addLayer(largest_mask, {'min': 0, 'max': 1, 'palette': ['red']}, 'Largest cluster (count == T21_max)')
print('T21_max:', T21_max.getInfo())
print("Min and Max values:", minMax.getInfo())

T21_max: 254
Min and Max values: {'T21_max': 254, 'T21_min': 2}


In [19]:

# Ensure a geemap Map object exists (create one if needed)
try:
    m  # noqa: F821
except NameError:
    # Use the existing import from earlier cells; do not re-import
    m = geemap.Map(center=[0, 0], zoom=2)

# Polygonize the largest masked cluster and zoom to it
vectors = largest_mask.reduceToVectors(
    reducer=ee.Reducer.countEvery(),
    geometry=geometry,
    scale=1000,
    maxPixels=1e9,
    eightConnected=True,
    geometryType='polygon'
)

# Compute area for each polygon and select the largest.
# Specify a non-zero maxError to avoid Geometry.area errors in server-side ops.
def set_area(feat):
    geom = feat.geometry()
    # use a small non-zero maxError (in meters)
    area_m = geom.area(1)
    return feat.set('area', area_m)

vectors = vectors.map(set_area)
largest_feature = ee.Feature(vectors.sort('area', False).first())

# Compute and print the area of the largest_feature
area_m = largest_feature.geometry().area(1)  # area in square meters (server-side)
area_m_val = area_m.getInfo()  # fetch to client
print('Largest feature area (m²):', area_m_val)
print('Largest feature area (km²):', area_m_val / 1e6)

# Add layers and center the map on the largest polygon
m.addLayer(vectors, {'color': 'orange'}, 'Largest cluster polygons')
m.addLayer(ee.FeatureCollection(largest_feature), {'color': 'red'}, 'Largest polygon')
m.centerObject(largest_feature, 10)
m

Largest feature area (m²): 254000801.57864597
Largest feature area (km²): 254.00080157864596


Map(bottom=9559.0, center=[-22.726087794490084, 121.72215718313251], controls=(WidgetControl(options=['positio…

In [20]:
import os
import json
from datetime import date

# Fetch the server-side feature to the client as a GeoJSON-like dict
feat_geojson = largest_feature.getInfo()

# Pretty-print GeoJSON to console for copy-paste
geojson_str = json.dumps(feat_geojson, indent=2)
print(geojson_str)

# Ensure output directory exists and write the GeoJSON file using the naming convention
out_dir = "./output"
os.makedirs(out_dir, exist_ok=True)
filename = f"{date.today().isoformat()}_largest_fire_global.geojson"
out_path = os.path.join(out_dir, filename)

with open(out_path, "w") as f:
    f.write(geojson_str)

print(f"\nWritten to: {out_path}")

{
  "type": "Feature",
  "geometry": {
    "geodesic": false,
    "type": "Polygon",
    "coordinates": [
      [
        [
          121.74661698467742,
          -22.711849836001676
        ],
        [
          121.73862069799252,
          -22.7028566300782
        ],
        [
          121.74836923152664,
          -22.7028566300782
        ],
        [
          121.72440274271153,
          -22.675877012316768
        ],
        [
          121.70490951369534,
          -22.675877012316768
        ],
        [
          121.68098327042904,
          -22.64889739454635
        ],
        [
          121.64200447684061,
          -22.64889739454635
        ],
        [
          121.66592305567193,
          -22.675877012316768
        ],
        [
          121.67566967018003,
          -22.675877012316768
        ],
        [
          121.68365125552208,
          -22.684870218240242
        ],
        [
          121.64466224008487,
          -22.684870218240242
        ],
 

## 2. Search and Request Planet Imagery for the Largest Fire

In this section we'll use the AOI derived from the previous section (the server-side feature stored in `largest_feature`) to search Planet's catalog and request high-resolution PlanetScope imagery for detailed inspection.

Planned steps
- Export or convert `largest_feature` to a client-side GeoJSON geometry (or buffer it slightly to create a practical AOI).
- Authenticate to Planet (provide your Planet API key via environment variable or the SDK).
- Build a Planet search using:
    - geometry filter from the AOI
    - date range (e.g., last 7–14 days)
    - cloud-cover filter (e.g., < 20%)
    - item type(s): `PSScene` / `PSScene4Band` (PlanetScope)
- Inspect search results (scene footprint, acquisition date, cloud cover) and pick candidate scenes.
- Place an order using the Planet Orders API (specify item ids, products/bundles, and delivery options).
- Monitor the order and, when ready, visualize delivered assets directly in the notebook (using geemap or by downloading and displaying the images).

Notes and tips
- Keep searches and orders small for quicker turnaround (limit to a few target items).
- Use a BBox with a small buffer around `largest_feature` to ensure full scene coverage.
- Store your Planet API key securely (do not hard-code it in the notebook). Example: export PL_API_KEY in your shell or use a notebook-based prompt to inject it at runtime.
- The next cells will provide concrete code examples for: converting the EE feature to GeoJSON, authenticating to Planet, running a filtered search, and submitting an order.